In [1]:
import re
import os
import time
import datetime
import random
import requests
import numpy as np
import pandas as pd
from datetime import datetime
from os import system   
from math import floor
from copy import deepcopy
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## Begin Article Finding

In [15]:
# Initialize driver and load website
driver = webdriver.Chrome()
driver.get('https://www.cnn.com/search?q=Hong%20Kong%20Protest&size=10&type=article')

In [ ]:
# Locate search
# search_button = driver.find_element_by_class_name('search-icon')
# search_button.click()

In [ ]:
# Input search term
# input_bar = driver.find_element_by_css_selector('input')
# input_bar.send_keys('Hong Kong Protest')

# Retrive search results
# send_button = driver.find_element_by_xpath('//*[@id="header-nav-container"]/div/div[2]/div/div[1]/form/button')
# send_button.click()

## Scrape Articles

In [92]:
# Function to scrape all urls of topic
def scrape_urls_cnn(urls, dates, start, stop):
    time.sleep(2)
    xpath = '/html/body/div[5]/div[2]/div/div[2]/div[2]/div/div[3]/div[{}]/div[2]/div[1]/span[2]'
    a_xpath = '/html/body/div[5]/div[2]/div/div[2]/div[2]/div/div[3]/div[{}]/div[2]/h3/a'
    ymd_start = datetime(start[0], start[1], start[2]).date()
    ymd_stop = datetime(stop[0], stop[1], stop[2]).date()
    
    # Scrape urls of each page
    for i in range(10):
        try:
            ele = driver.find_element_by_xpath(xpath.format(i+1)).text.replace(',', '')
            ts = datetime.strptime(ele, '%b %d %Y').date()
            href = driver.find_element_by_xpath(a_xpath.format(i+1)).get_attribute('href')
            if ts < ymd_start:
                print('All URLS in date range scraped')
                print(f'Number of article URLs: {len(urls)}\n')
                return urls, dates
            elif ts < ymd_stop:
                if re.search('(live-news)', href):
                    continue
                else:
                    urls.append(href)
                    dates.append(ts)
        except:
            print('All URLS in date range scraped')
            print(f'Number of article URLs: {len(urls)}\n')
            return urls, dates
    
    # Click to next page if there is one
    try:
        np_xpath= '/html/body/div[5]/div[2]/div/div[2]/div[2]/div/div[5]/div/div[3]'
        driver.find_element_by_xpath(np_xpath).click()
    except:
        print('No more pages')
        print(f'Number of article URLs: {len(urls)}\n')
        return urls, dates

    return scrape_urls_cnn(urls, dates, start, stop)

In [85]:
def clean_body(article):
    cleaned = re.sub(r"^.*?\)|(CNN's.*)", "", article)
    return cleaned

In [86]:
# Function to scrape headline, byline, and article body
def scrape_one_cnn(url):
    try:
        driver.get(url)
        time.sleep(2)
        headline = driver.find_element_by_class_name('pg-headline').text
        texts = driver.find_elements_by_class_name('zn-body__paragraph')
        article = ' '.join([ text.text for text in texts ])
        article = clean_body(article)
        byline = driver.find_element_by_class_name('metadata__byline__author').text
    except:
        headline = 'No headline'
        byline = 'No byline'
        article = 'No text'
    return headline, byline, article     

In [87]:
# Function to scrape all designated articles
def scrape_articles_cnn(start, stop):
    urls = []
    dates = []
    bodies = []
    headlines = []
    bylines = []
    urls, dates = scrape_urls_cnn(urls, dates, start, stop)
    for url in urls:
        headline, byline, article = scrape_one_cnn(url)
        bodies.append(article)
        headlines.append(headline)
        bylines.append(byline)
    return urls, dates, headlines, bylines, bodies      

In [88]:
def scrape_cnn(start, stop):
    urls = []
    dates = []
    headlines = []
    bylines = []
    bodies = []
    
    urls, dates, headlines, bylines, bodies = scrape_articles_cnn(start, stop)
    df = pd.DataFrame()
    df['url'] = urls
    df['date'] = dates
    df['headline'] = headlines
    df['byline'] = bylines
    df['body'] = bodies
    df['source'] = 'CNN'
    df['index'] = range(len(df.index))
    df.set_index('index', inplace=True)
    
    return df
    

In [94]:
driver = webdriver.Chrome()
driver.get('https://www.cnn.com/search?q=Hong%20Kong%20Protest&size=10&type=article')

In [95]:
df = scrape_cnn(start=(2019, 1, 1), stop=(2020, 1, 1))

All URLS in date range scraped
Number of article URLs: 4



In [96]:
df

,url,date,headline,byline,body,source
index,,,,,,
0,https://www.cnn.com/2019/12/30/world/2010s-end...,2019-12-30,The 2010s were relentless. Here are some of th...,"By Madeline Holcombe and Christina Maxouris, CNN","When the clock strikes midnight on January 1, ...",CNN
1,https://www.cnn.com/style/article/biggest-fash...,2019-12-17,No headline,No byline,No text,CNN
2,https://www.cnn.com/2019/12/27/tech/grand-thef...,2019-12-27,Grand Theft Auto V becomes latest battleground...,"Isaac Yee, CNN Business",The Hong Kong political crisis is now playing ...,CNN
3,https://www.cnn.com/2019/12/25/asia/asia-youth...,2019-12-25,Young people across Asia pushed for change in ...,"By Jessie Yeung, Erin Chan and Michelle Lim, CNN",2019 was the year of youth activism. Around th...,CNN


In [83]:
df.iloc[0]['body']

'When the clock strikes midnight on January 1, an unparalleled decade comes to a close: one that saw everything from NASA\'s first all-female spacewalk to the aftermath of natural disasters and the death of Osama bin Laden. There were times of real change and hope. The White House was lit up in rainbow colors when the Supreme Court struck down same-sex marriage bans. The world watched in awe as a Thai soccer team and their coach were rescued one-by-one by a group of brave diving experts after being trapped for more than three weeks. And there were moments so shocking we were left feeling helpless. Twenty-six people were killed in a shooting at Sandy Hook Elementary School and eight parishioners and their pastor were gunned down during Bible study at Emanuel African Methodist Episcopal Church. Revolutionary moments made history in real time as anger and passion turned into movements -- like the one made up of thousands of students who skipped classes worldwide to demand action from thei

In [97]:
driver.quit()